# 1. Tiền xử lý dữ liệu

## 1.1 Chuẩn hóa dữ liệu

In [5]:
import os, sys, json, re, unicodedata, html, string, random
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

random.seed(42)
np.random.seed(42)

In [7]:
PROJECT_ROOT = Path(r"D:\Projects\Tiny Transforner for Machine Translation")
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
MODEL_DIR = PROJECT_ROOT / "models"
REPORTS_DIR = PROJECT_ROOT / "reports"
for d in (RAW_DIR, PROCESSED_DIR, MODEL_DIR, REPORTS_DIR):
    d.mkdir(parents=True, exist_ok=True)

RAW_EN = RAW_DIR / "train_en.txt"
RAW_VI = RAW_DIR / "train_vi.txt"
print("PROJECT_ROOT:", PROJECT_ROOT)
print("RAW_EN:", RAW_EN)
print("RAW_VI:", RAW_VI)

PROJECT_ROOT: D:\Projects\Tiny Transforner for Machine Translation
RAW_EN: D:\Projects\Tiny Transforner for Machine Translation\data\raw\train_en.txt
RAW_VI: D:\Projects\Tiny Transforner for Machine Translation\data\raw\train_vi.txt


In [10]:
#Kiểm tra dữ liệu
def read_lines_utf8(path: Path):
    text = path.read_text(encoding="utf-8")
    return text.splitlines()

en_raw = read_lines_utf8(RAW_EN)
vi_raw = read_lines_utf8(RAW_VI)

In [11]:
assert len(en_raw) == len(vi_raw), f"Length mismatch EN={len(en_raw)} vs VI={len(vi_raw)}"
assert all(line is not None for line in en_raw) and all(line is not None for line in vi_raw)

print(f"Num pairs: {len(en_raw)}")
print("Preview (first 5 lines):")
for i in range(min(5, len(en_raw))):
    print(f"[{i}] EN: {en_raw[i]}")
    print(f"    VI: {vi_raw[i]}")

Num pairs: 41
Preview (first 5 lines):
[0] EN: Banh mi, pho, iced coffee and Hoi An cao lau noodles have earned high marks from international travel magazines in 2022, reaffirming Vietnam’s growing reputation as a global culinary attraction. 
    VI: Bánh mì, phở, cà phê đá và cao lầu Hội An đã giành được điểm cao từ các tạp chí du lịch quốc tế vào năm 2022, tái khẳng định danh tiếng ngày càng tăng của Việt Nam như một điểm thu hút ẩm thực toàn cầu. 
[1] EN: In January, CNN included pho bo (Vietnamese beef noodle soup) in its list of the 20 best soups in the world, along with China’s Lanzhou noodle soup, and Thailand’s tom yum goong.
    VI: Vào tháng 1, CNN đã đưa phở bò vào danh sách 20 món canh ngon nhất thế giới, cùng với phở Lan Châu của Trung Quốc và tom yum goong của Thái Lan.
[2] EN: Till now, pho bo remains the most beloved version of the dish in Vietnam and has become the most popular breakfast in the country, with options that include the original raw beef, a mix of raw and 

In [ ]:
#Làm sạch dữ liệu

#Regex tiền biên dịch
RE_CONTROL      = re.compile(r"[\u0000-\u001F\u007F]", flags=re.UNICODE)
RE_ZW           = re.compile(r"[\u200B-\u200F\u202A-\u202E\u2060-\u206F\uFEFF]")
RE_HTML_TAG     = re.compile(r"<[^>]+>")
RE_MULTISPACE   = re.compile(r"\s+")
RE_SP_BEFORE_P  = re.compile(r"\s+([,.:;!?])")
RE_SPACE_POPEN  = re.compile(r"\(\s+")
RE_SPACE_PCLOSE = re.compile(r"\s+\)")
RE_MULTI_DOTS   = re.compile(r"\.{3,}")           # ellipsis
RE_MULTI_DASH   = re.compile(r"[–—−]+")           # dash variants
RE_QUOTE_CHARS  = re.compile(r"[“”«»„‟‚‛]")        # thành "